# Frame Overlap Tutorial (v0.2.0)

This tutorial demonstrates the new object-oriented API introduced in version 0.2.0 for analyzing neutron Time-of-Flight frame overlap data.

## Overview

The frame_overlap package now provides three main classes:

1. **Data**: Load and process ToF data with convolution, frame overlap, and Poisson sampling
2. **Reconstruct**: Apply deconvolution filters to reconstruct original signals
3. **Analysis**: Fit reconstructed data to extract material parameters

We'll walk through a complete workflow from data loading to material analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from frame_overlap import Data, Reconstruct, Analysis, ParametricScan

# Set up plotting
plt.style.use('default')
%matplotlib inline

## 1. Data Loading and Processing

First, let's load neutron ToF data. The `Data` class handles both signal and openbeam files.

In [ ]:
# Load data from CSV files
data = Data(signal_file='iron_powder.csv',
            openbeam_file='openbeam.csv',
            flux=1e6,           # neutrons/s
            duration=3600,      # seconds
            threshold=30)       # filter stacks < 30

print(data)
print(f"\nLoaded {len(data.table)} data points")
print(f"Time range: {data.table['time'].min():.1f} - {data.table['time'].max():.1f} µs")

### Visualize the raw data

In [ ]:
# Plot signal
fig = data.plot()
plt.show()

# Compare signal with openbeam
fig = data.plot_comparison()
plt.show()

## 2. Instrument Response Convolution

Simulate the effect of measuring with an instrument that has a finite pulse duration.

In [ ]:
# Store original for comparison
data_original = data.copy()

# Apply square response convolution (200 µs pulse)
data.convolute_response(pulse_duration=200)

print("Convolution applied")
print(f"Data shape: {data.table.shape}")

## 3. Frame Overlap Creation

Create overlapping frames with a custom time sequence. The sequence `[0, 12, 10, 25]` means:
- Frame 1 starts at t = 0 ms
- Frame 2 starts at t = 12 ms
- Frame 3 starts at t = 12 + 10 = 22 ms
- Frame 4 starts at t = 22 + 25 = 47 ms

In [ ]:
# Create frame overlap
data.overlap(seq=[0, 12, 10, 25])

print(f"Frame overlap created with kernel: {data.kernel}")
print(f"New data length: {len(data.table)}")

# Plot overlapped data
fig = data.plot()
plt.title('Data after Frame Overlap')
plt.show()

## 4. Poisson Sampling

Apply Poisson counting statistics to simulate realistic measurement conditions.

In [ ]:
# Apply Poisson sampling with 80% duty cycle
data.poisson_sample(duty_cycle=0.8)

# Plot final processed data
fig = data.plot()
plt.title('Data after Poisson Sampling')
plt.show()

## 5. Signal Reconstruction

Now we'll use the `Reconstruct` class to deconvolve the overlapped frames and recover the original signal.

In [ ]:
# Create Reconstruct object
recon = Reconstruct(data)

# Apply Wiener filter
recon.filter(kind='wiener', noise_power=0.01)

print("Reconstruction complete")
print(f"\nReconstruction Statistics:")
for key, value in recon.get_statistics().items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

### Visualize reconstruction results

In [ ]:
# Plot reconstructed signal
fig = recon.plot_reconstruction()
plt.show()

# Compare with reference (if available)
try:
    fig = recon.plot_comparison()
    plt.show()
    
    fig = recon.plot_residuals()
    plt.show()
except ValueError:
    print("Reference data not available for comparison")

### Try different deconvolution methods

In [ ]:
# Compare Wiener, Richardson-Lucy, and Tikhonov methods
methods = ['wiener', 'lucy', 'tikhonov']
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, method in zip(axes, methods):
    recon_test = Reconstruct(data)
    recon_test.filter(kind=method, noise_power=0.01, iterations=10)
    
    ax.plot(recon_test.reconstructed_table['time'],
            recon_test.reconstructed_table['counts'],
            label=method.capitalize())
    ax.set_xlabel('Time (µs)')
    ax.set_ylabel('Counts')
    ax.set_title(f'{method.capitalize()} Filter')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Material Analysis

Fit the reconstructed data to extract material parameters like thickness and composition.

In [ ]:
# Create Analysis object
analysis = Analysis(recon)

# Set material composition (default is Fe-alpha + 4% Cellulose)
analysis.set_cross_section(['Fe_alpha', 'Cellulose'], [0.96, 0.04])

# Fit with square response function
try:
    analysis.fit(response='square')
    
    # Print fit report
    print(analysis.get_fit_report())
except Exception as e:
    print(f"Fitting failed: {e}")
    print("This can happen with random test data")

### Visualize fit results

In [ ]:
try:
    # Plot fit
    fig = analysis.plot_fit()
    plt.show()
    
    # Plot residuals
    fig = analysis.plot_residuals()
    plt.show()
    
    # Plot material composition
    fig = analysis.plot_materials()
    plt.show()
except (ValueError, AttributeError):
    print("Fit visualization not available")

## 7. Parametric Scans

Explore how different processing parameters affect the reconstruction and fitting results.

In [ ]:
# Load fresh data for scanning
scan_data = Data(signal_file='iron_powder.csv', threshold=30)

# Create parametric scan
scan = ParametricScan(scan_data)

# Add parameters to scan
scan.add_parameter('pulse_duration', [100, 200, 300])
scan.add_parameter('n_frames', [2, 3, 4])

print("Running parametric scan...")
print(f"Total combinations: {3 * 3} = 9")

# Run scan (may take a minute)
try:
    scan.run(verbose=True)
    
    # Get results
    results = scan.get_results()
    print(f"\nSuccessful runs: {len(results)} / 9")
    print("\nSample results:")
    print(results[['pulse_duration', 'n_frames', 'recon_r_squared', 'fit_thickness']].head())
except Exception as e:
    print(f"Scan failed: {e}")

### Visualize scan results

In [ ]:
try:
    # Parameter sensitivity plot
    fig = scan.plot_parameter_sensitivity('pulse_duration', 'recon_r_squared', groupby='n_frames')
    plt.show()
    
    # 2D heatmap
    fig = scan.plot_heatmap('pulse_duration', 'n_frames', 'fit_thickness')
    plt.show()
    
    # Summary statistics
    fig = scan.plot_summary()
    plt.show()
except Exception as e:
    print(f"Plotting failed: {e}")

## 8. Method Chaining Example

The new API supports fluent method chaining for concise workflows.

In [ ]:
# Complete workflow in a chained style
data_chain = (Data(signal_file='iron_powder.csv', threshold=30)
              .convolute_response(pulse_duration=200)
              .overlap(seq=[0, 12, 10, 25])
              .poisson_sample(duty_cycle=0.8))

print("Data processing complete via method chaining")
print(data_chain)

# Continue with reconstruction
recon_chain = Reconstruct(data_chain).filter(kind='wiener', noise_power=0.01)
print(f"\nReconstruction χ²/dof: {recon_chain.statistics.get('chi2_per_dof', 'N/A')}")

## Summary

This tutorial covered:

1. **Data Loading**: Loading signal and openbeam ToF data
2. **Convolution**: Applying instrument response functions
3. **Frame Overlap**: Creating overlapping frame sequences
4. **Poisson Sampling**: Simulating counting statistics
5. **Reconstruction**: Deconvolving signals with multiple methods
6. **Analysis**: Fitting for material parameters
7. **Parametric Scans**: Exploring parameter sensitivity
8. **Method Chaining**: Concise workflow syntax

The new v0.2.0 API provides a clean, intuitive interface for neutron ToF frame overlap analysis!

## Additional Resources

- Documentation: See README.md for complete API reference
- Legacy API: The v0.1.0 functional API remains available for backward compatibility
- Source code: https://github.com/TsvikiHirsh/frame_overlap